<a href="https://colab.research.google.com/github/andre-arantes/ia/blob/master/tp1/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Previsão de Diabetes

Este projeto tem como objetivo prever a probabilidade de uma pessoa ter diabetes com base em vários fatores, como idade, nível de HbA1c, nível de glicose no sangue e gênero. O conjunto de dados usado neste projeto é o Diabetes Prediction Dataset.

Usaremos o algoritmo Random Forest Classifier para treinar nosso modelo e prever a probabilidade de diabetes. Também usaremos várias técnicas de pré-processamento de dados, como subamostragem, para melhorar a precisão do nosso modelo.

In [ ]:
# Importando bibliotecas
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
# Carregando os dados
base = pd.read_csv('/content/sample_data/diabetes_prediction_dataset.csv')

# Ler dados da base

In [ ]:
base.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
# checar existencias de campos nulos
base.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [ ]:
# checar natureza dos dados
for i in base.columns:
    print(base[i].value_counts())
    print('\n')

Female    58552
Male      41430
Other        18
Name: gender, dtype: int64


80.00    5621
51.00    1619
47.00    1574
48.00    1568
53.00    1542
         ... 
0.48       83
1.00       83
0.40       66
0.16       59
0.08       36
Name: age, Length: 102, dtype: int64


0    92515
1     7485
Name: hypertension, dtype: int64


0    96058
1     3942
Name: heart_disease, dtype: int64


No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: smoking_history, dtype: int64


27.32    25495
23.00      103
27.12      101
27.80      100
24.96      100
         ...  
58.23        1
48.18        1
55.57        1
57.07        1
60.52        1
Name: bmi, Length: 4247, dtype: int64


6.6    8540
5.7    8413
6.5    8362
5.8    8321
6.0    8295
6.2    8269
6.1    8048
3.5    7662
4.8    7597
4.5    7585
4.0    7542
5.0    7471
8.8     661
8.2     661
9.0     654
7.5     643
6.8     642
7.0     634
Name: HbA1c_level, dtype: int6

**Separando os atributos de entrada e de classe**

In [ ]:
X = base.copy()
y = X.pop('diabetes')

#Pré-processamento dos dados

In [ ]:
# remover campo 'smoking_history' devido a quantidade de informações faltantes
X.drop('smoking_history',axis=1,inplace=True)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder_gender = LabelEncoder()
X.iloc[:,0] = label_encoder_gender.fit_transform(X.iloc[:,0])


<ipython-input-40-2778dbb9c2a5>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:,0] = label_encoder_gender.fit_transform(X.iloc[:,0])


**Divisão dos dados em treino e teste**

foi feita a divisão dos dados em 80\% para treino e 20\% para teste.

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.20, random_state = 42)

**Balanciamento dos dados**

Para o balanceamento de dados, foi verificado que haviam muitas instâncias da classe majoritária, então foi decidido implementar o undersampling na base de treino.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
# balanceia a base de dados de forma que a base majoritaria fique 40% maior
us=RandomUnderSampler(sampling_strategy=0.4)
X_treino,y_treino = us.fit_resample(X_treino,y_treino)

**Examine o Random Forest Padrão para Determinar Parâmetros.**

In [ ]:
modelo = RandomForestClassifier(random_state = 0)
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(modelo.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


#Random Search com Cross Validation

Para montagem do RandomGrid, foi utilizado os seguintes parametros: quantidade de árvores (40 a 400), cálculo para quantidade de atributos (log2 ou sqrt), profundidade máxima da árvore (1 a 20 ou sem limite), mínimo de samples para dividir um nó (2 3 ou 5), numero mínimo de samples em cada folha da árvore (1, 2 ou 4) e uso de bootstrap (verdadeiro ou falso).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 40, stop = 400, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 20, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [1, 10, 20, None],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [40, 80, 120, 160, 200, 240, 280, 320, 360, 400]}


**RandomSerach com cross validation**

RandomizedSearchCV com o RandomGrid como um de seus parâmetros, além do kfold (5) percorrendo 100 combinações e com o scoring buscando o melhor recall da classe 1.

In [ ]:

from sklearn.metrics import make_scorer, recall_score

# Create a custom scoring function for recall of class 1
scoring = {'Recall_Class1': make_scorer(recall_score, pos_label=1)}

# Random search of parameters, using 5 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=modelo, param_distributions=random_grid,
                              n_iter = 100, scoring=scoring,refit='Recall_Class1',
                              cv = 5, verbose=2, random_state=0, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_treino, y_treino)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 10, 20, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [40, 80, 120, 160, 200,
                                                         240, 280, 320, 360,
                                                         400]},
                   random_state=0, refit='Recall_Class1',
                   return_train_score=True,
                   scoring={'Recall_Class1': make_scorer(recall_score, pos_label=1)},
                   verbose=2)

**Avaliação do modelo base**

A avaliação do modelo é feita a partir do recall da classe verdadeira (tem diabetes)

In [ ]:
def evaluate(model, test_features, test_labels):
    previsoes = model.predict(test_features)
    cr = classification_report(test_labels, previsoes, output_dict=True)['1']['recall']
    return cr

In [ ]:
base_model = RandomForestClassifier(random_state = 0)
base_model.fit(X_treino, y_treino)
base_recall = evaluate(base_model, X_teste, y_teste)
base_recall

0.8079625292740047

**avaliação do melhor modelo**

In [ ]:
best_random = rf_random.best_estimator_
random_recall = evaluate(best_random, X_teste, y_teste)
random_recall

0.810304449648712

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_recall - base_recall) / base_recall))

Improvement of 0.29%.


In [ ]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': None,
 'bootstrap': False}

In [ ]:
rf_random.cv_results_

{'mean_fit_time': array([ 1.85082049,  1.49434938,  7.29233088,  2.81583734,  0.6981133 ,
         4.49384947,  8.51577015,  6.0908649 ,  7.3856997 ,  9.114856  ,
         8.66757312,  2.7173214 ,  1.11781611,  5.95895095,  1.08361015,
         6.46036763,  5.83845782,  4.21065879,  4.18525376,  9.11607208,
         2.12012796,  3.25963435,  2.10178242,  1.89474683, 11.33501606,
         1.15990796,  0.83608885,  3.5142437 ,  4.08258839,  2.90149059,
         0.65375047,  1.83778081,  5.99505978,  7.02131362,  7.34572406,
         3.04848051,  7.80584078,  2.38140812,  2.1680469 ,  7.1583415 ,
         1.35293503,  1.106251  ,  7.00662909,  1.48223548,  8.22245941,
         6.72069321,  2.52071247,  6.68461823,  0.69824429,  1.51154795,
         2.65431452,  8.18744874,  0.52358384,  3.63030467,  0.57814951,
         5.34284587,  8.47615352,  4.21387582, 10.74303627,  4.09697313,
         1.44559817,  2.97245502,  3.62915816,  0.24252882,  2.17206335,
         6.92624211,  1.38932996, 

#Resultados

In [ ]:
previsoes = best_random.predict(X_teste)
print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96     18292
           1       0.59      0.81      0.69      1708

    accuracy                           0.94     20000
   macro avg       0.79      0.88      0.83     20000
weighted avg       0.95      0.94      0.94     20000

